In [62]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn import preprocessing, ensemble, cross_validation

In [55]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [58]:
# cast certain columns to float64 since sklearn treats int as categorical
def cast_to_float(df, col_names):
    for col in col_names:
        df[col] = df[col].astype(float, copy=False)

In [51]:
def encode_onehot(df):
    categorical_col_names = df.dtypes[df.dtypes == 'object'].index.values
    return pd.get_dummies(df, columns=categorical_col_names)

In [ ]:
def rf_regress(df_X, y, n_trees=200, loss_criterion='mse', min_node_size=5):    
    # loss_criterion could be mse or mae
    # num features used for a tree = num_features * 0.33
    num_features = df_X.columns.size
    rf_regr = ensemble.RandomForestRegressor(n_estimators=n_trees,
                                             criterion=loss_criterion,
                                             max_features=0.33,
                                             min_samples_leaf=min_node_size)
    rf_regr = rf_regr.fit(df_X, y)
    return rf_regr

In [73]:
need_float_col_names = ['LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                        'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
                        'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']